In [1]:
import numpy as np
import cv2
from sklearn.metrics import pairwise

In [2]:
# Region of interest, where the hand needs to be detected
roi_top = 30
roi_bottom = 300
roi_right = 300
roi_left = 600

# define background
background = None
accumulated_weight = 0.5


In [3]:
def calc_accum_avg(frame, accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    # computes weighted average, accumulates it and updates the background
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment(frame, threshold=25):
    global background
    
    # calculates an absolute difference between frame and background
    diff = cv2.absdiff(background.astype("uint8"), frame)

    # threshold the image to define the background
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # finding the contours
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # if the length of contours is 0 it means no hand was detected
    if len(contours) == 0:
        return None
    else:
        # the largest external contour is supposed to be the hand 
        hand_segment = max(contours, key=cv2.contourArea)
        
        # returns both the hand segment and the thresholded hand image
        return (thresholded, hand_segment)

In [5]:
def count_fingers(thresholded, hand_segment):
    
    # calculates the convex hull of the hand segment
    conv_hull = cv2.convexHull(hand_segment)
    
    # finds the top, bottom, left , and right points
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])

    # calculates the central point
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    # calculates the euclidean distance between the center point of the hand and the left, right, top, and bottom points
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    # takes the largest distance
    max_distance = distance.max()
    
    # creates a circle with 80% radius of the maximum euclidean distance
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)

    
    # draw the circular ROI    
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    
    # uses bit-wise AND with the cirle ROI as a mask.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # finds contours in circle ROI
    contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # starts counting at 0
    count = 0

    # loops through the contours to detect more fingers.
    for cnt in contours:
        
        # Bounding box of countour
        (x, y, w, h) = cv2.boundingRect(cnt)

        # increment count of fingers based if:
        
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        # 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        
        if  out_of_wrist and limit_points:
            count += 1

    return count

In [6]:
cam = cv2.VideoCapture(0)

# intialize a frame count
num_frames = 0

# keep looping, until it is interrupted
while True:

    # gets the current frame
    ret, frame = cam.read()

    # flips the frame so it is not shown as the mirror view
    frame = cv2.flip(frame, 1)

    # copies the frame
    frame_copy = frame.copy()

    # grabs the ROI from the frame
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]

    # grayscale and blur for ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # first 30 frames the average of the background is calculated
    if num_frames < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frames <= 59:
            cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Finger Count",frame_copy)
            
    else:
        
        # segment the hand region
        hand = segment(gray)

        # First check if we were able to actually detect a hand
        if hand is not None:
            
            # unpack
            thresholded, hand_segment = hand

            # draws contours around hand
            cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

            # counts the fingers
            fingers = count_fingers(thresholded, hand_segment)

            # displays count
            cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # displays the thresholded image
            cv2.imshow("Thesholded", thresholded)

    # draws ROI Rectangle on frame copy
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    # increment the number of frames for tracking
    num_frames += 1

    # displays the frame with segmented hand
    cv2.imshow("Finger Count", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()